In [1]:
import csv
import os
import random
import sklearn.model_selection

from shutil import copyfile
from PIL import ImageFile

import torch
import torchvision 
import torchvision.transforms as transforms

import torchvision.models as models
import torch.nn as nn
import scipy

import torch.optim as optim
from torch.optim import lr_scheduler

#-----
import numpy as np
import time
import copy

from PIL import Image
from torch.autograd import Variable
import torchvision.transforms as transforms

from glob import glob
import numpy as np
#-----

import torch
import torchvision.models as models
import torch.nn as nn
import torchvision.datasets
import torchvision.transforms as transforms
from PIL import ImageFile

import numpy as np

import torch.optim as optim
import time

In [2]:
TRAIN_TEST_CSV_PATH = '../C1-P1_Train Dev_fixed/train.csv'
TRAIN_CSV_PATH = '../C1-P1_Train Dev_fixed/train_split.csv'
VALID_CSV_PATH = '../C1-P1_Train Dev_fixed/dev.csv'

ORIGINAL_TRAIN_TEST_DATA_PATH = '../C1-P1_Train Dev_fixed/C1-P1_Train/' 
# ORIGINAL_TRAIN_DATA_PATH = '../C1-P1_Train Dev_fixed/C1-P1_Train/' 
ORIGINAL_VALID_DATA_PATH = '../C1-P1_Train Dev_fixed/C1-P1_Dev/' 


TRAIN_DATA_PATH = '../data/train'
VALID_DATA_PATH = '../data/valid'
TEST_DATA_PATH = '../data/test'

TRAINED_MODEL_WEIGHTS_FILE_NAME = 'model_v4_weights_run1.pt'
NUM_TEST_DATASET = 100

image_size = 299
# image_size = 224

In [3]:
nb_start = time.time()

In [4]:
# Hyperparameters

batch_size = 16
lr = 0.001
valid_loss_stable_count = 14
num_training_epochs = 100
exp_lr_scheduler_step_size = 7
exp_lr_scheduler_gamma = 0.1
sgd_momentum = 0.9

# We can choose not to use  data in "../AIMango_sample/" as test dataset

In [5]:
# # use data in "../AIMango_sample/" as test dataset for evaluation
# # the format of this dataset is different from the rest
# # the csv file looks like: D-Plant2_0610_3.jpg,等級B
# # the following code is to transform it into: D-Plant2_0610_3.jpg,B

# TEST_CSV_PATH = '../AIMango_sample/label.csv'
# ORIGINAL_TEST_DATA_PATH = '../AIMango_sample/sample_image/' 

# new_data = []
# with open(TEST_CSV_PATH) as f:
#     reader = csv.reader(f, delimiter=',')
#     for row in reader:
#         new_label = row[1][len(row[1])-1] # remove 等級 from the label 等級B and get the new lable B
#         new_data.append([row[0],new_label])


# # save the data from the original csv file that was transformed by the code above to a new csv file
# folder, filename = os.path.split(TEST_CSV_PATH)
# TEST_CSV_PATH = os.path.join(folder, 'label_new.csv')
                                 
# with open(TEST_CSV_PATH, 'w') as f:

#     writer = csv.writer(f)
    
#     for row in new_data:
#         writer.writerow(row)

# Split data from train dataset. get last 100 records as test dataset

In [6]:
new_train_data = []
new_test_data = []
all_data = []

TEST_CSV_PATH = '../C1-P1_Train Dev_fixed/test_split.csv'

with open(TRAIN_TEST_CSV_PATH) as csv_file:

    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
            continue  #header
        all_data.append(row)
        
        line_count += 1
        
indices = list(range(len(all_data)))

random.shuffle(indices)
training_dataset, test_dataset = sklearn.model_selection.train_test_split(indices, train_size=len(all_data)-NUM_TEST_DATASET, test_size=NUM_TEST_DATASET)

for idx in training_dataset:
    new_train_data.append(all_data[idx])
for idx in test_dataset:
    new_test_data.append(all_data[idx])  

with open(TRAIN_CSV_PATH, 'w') as f:

    writer = csv.writer(f)
    
    writer.writerow(['image_id','label'])
    for row in new_train_data:
        writer.writerow(row)
        
with open(TEST_CSV_PATH, 'w') as f:

    writer = csv.writer(f)
    
    writer.writerow(['image_id','label'])
    for row in new_test_data:
        writer.writerow(row)

In [7]:
# parse csv file and make the folder structure for pytorch test dataset
def prepare_file_structure_for_pytoch(csv_path, src_data_path, dst_data_path):
    with open(csv_path) as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                line_count += 1
                continue  #header

            src_path = os.path.join(src_data_path, row[0])
            dest_path = os.path.join(dst_data_path, row[1], row[0])
            dest_folder_path = os.path.join(dst_data_path, row[1])
            if not os.path.exists(dest_folder_path):
                os.makedirs(dest_folder_path)
            if not os.path.isfile(dest_path):
                copyfile(src_path, dest_path)
        
            line_count += 1

In [8]:
!rm -rf ../data/train
!rm -rf ../data/test
!rm -rf ../data/valid
# make file structure for training dataset
prepare_file_structure_for_pytoch(TRAIN_CSV_PATH, ORIGINAL_TRAIN_TEST_DATA_PATH, TRAIN_DATA_PATH)
    
# make file structure for validation dataset
prepare_file_structure_for_pytoch(VALID_CSV_PATH, ORIGINAL_VALID_DATA_PATH, VALID_DATA_PATH)

# make file structure for test dataset
prepare_file_structure_for_pytoch(TEST_CSV_PATH, ORIGINAL_TRAIN_TEST_DATA_PATH, TEST_DATA_PATH)

In [9]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
aug_transform = transforms.Compose([
                                transforms.Resize(image_size),
                                transforms.CenterCrop(image_size),
                                transforms.RandomHorizontalFlip(p=0.5),
                                transforms.RandomRotation(degrees=(-15, 15)),
                                transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
                                transforms.ToTensor(),
                                transforms.Normalize(
                                    mean=(0.485, 0.456, 0.406),
                                    std =(0.229, 0.224, 0.225))
                               ])

vanila_transform = transforms.Compose([
                                transforms.Resize(image_size),
                                transforms.CenterCrop(image_size),
                                transforms.ToTensor(),
                                transforms.Normalize(
                                    mean=(0.485, 0.456, 0.406),
                                    std =(0.229, 0.224, 0.225))
                               ])

loaders = {}
datasets = {}

datasets['train'] = torchvision.datasets.ImageFolder(TRAIN_DATA_PATH, transform=aug_transform)
loaders['train'] = torch.utils.data.DataLoader(datasets['train'],
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=1)

datasets['valid'] = torchvision.datasets.ImageFolder(VALID_DATA_PATH, transform=vanila_transform)
loaders['valid'] = torch.utils.data.DataLoader(datasets['valid'],
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=1)
# datasets['test'] = torchvision.datasets.ImageFolder(TEST_DATA_PATH, transform=vanila_transform)
# loaders['test'] = torch.utils.data.DataLoader(datasets['test'],
#                                           batch_size=batch_size,
#                                           shuffle=True,
#                                           num_workers=1)

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [11]:
#MODEL in evaluation

model = models.inception_v3(pretrained=True)

model.aux_logits=False

num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 3)

model = model.to(device)

In [12]:
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=sgd_momentum)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=exp_lr_scheduler_step_size, gamma=exp_lr_scheduler_gamma)

In [13]:
# train the model
def train(n_epochs, loaders, model, optimizer, criterion, scheduler, save_path):

    best_acc = 0.0 
    valid_loss_min = np.Inf 
    valid_loss_stable_counter = 0

    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
                
        start = time.time()
        
        ###################
        # train the model #
        ###################
        scheduler.step()
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            data, target = data.to(device), target.to(device)
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update accumulated training loss
            train_loss += loss.item()*data.size(0)
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        valid_corrects = 0
        
        for batch_idx, (data, target) in enumerate(loaders['valid']):

            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            _, preds = torch.max(output, 1)
            
            # calculate the batch loss
            loss = criterion(output, target)
            # update accumulated validation loss 
            valid_loss += loss.item()*data.size(0)
    
            # accumulate number of the accurate predictions
            valid_corrects += torch.sum(preds == target.data)
        
        train_loss = train_loss/len(loaders['train'].dataset)
        valid_loss = valid_loss/len(loaders['valid'].dataset)
        
        epoch_acc = valid_corrects.double() / len(loaders['valid'].dataset)
        
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tValidation Accuracy: {:.4f} \ttime: {:.1f}'.format(
            epoch, 
            train_loss,
            valid_loss,
            epoch_acc,
            time.time() - start
            ))
        
#         if epoch_acc > best_acc:
#             print('Accuracy increased from {:.4f} to {:.4f}. Model was saved'.format(
#                 best_acc,
#                 epoch_acc
#             ))

#             best_acc = epoch_acc
#             best_model_wts = copy.deepcopy(model.state_dict())
#             torch.save(best_model_wts, save_path)

        if valid_loss < valid_loss_min:
            print('Validation loss decreased from {:.6f} to {:.6f}. Model was saved'.format(
                valid_loss_min,
                valid_loss
            ))

            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, TRAINED_MODEL_WEIGHTS_FILE_NAME)


            valid_loss_min = valid_loss
            
            valid_loss_stable_counter = 0
        else:
            valid_loss_stable_counter += 1
            if valid_loss_stable_counter >= valid_loss_stable_count:
                # early stop
                return model
    
    # return trained model
    return model

model = train(num_training_epochs,
                       loaders,
                       model,
                       optimizer,
                       criterion,
                       exp_lr_scheduler,
                       TRAINED_MODEL_WEIGHTS_FILE_NAME)

Epoch: 1 	Training Loss: 0.730522 	Validation Loss: 0.486846 	Validation Accuracy: 0.8075 	time: 260.3
Validation loss decreased from inf to 0.486846. Model was saved
Epoch: 2 	Training Loss: 0.552391 	Validation Loss: 0.435703 	Validation Accuracy: 0.8187 	time: 256.2
Validation loss decreased from 0.486846 to 0.435703. Model was saved
Epoch: 3 	Training Loss: 0.510847 	Validation Loss: 0.440340 	Validation Accuracy: 0.7975 	time: 261.6
Epoch: 4 	Training Loss: 0.467503 	Validation Loss: 0.440017 	Validation Accuracy: 0.8025 	time: 271.0
Epoch: 5 	Training Loss: 0.452827 	Validation Loss: 0.432475 	Validation Accuracy: 0.8175 	time: 267.6
Validation loss decreased from 0.435703 to 0.432475. Model was saved
Epoch: 6 	Training Loss: 0.400968 	Validation Loss: 0.434913 	Validation Accuracy: 0.8075 	time: 259.3
Epoch: 7 	Training Loss: 0.385185 	Validation Loss: 0.484399 	Validation Accuracy: 0.7700 	time: 256.7
Epoch: 8 	Training Loss: 0.328099 	Validation Loss: 0.435609 	Validation Accu

# The following module can be run separately if trained weights are available

### The block below define the Model and should be exactly the same as the one above

In [14]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [15]:
model = models.inception_v3(pretrained=True)

model.aux_logits=False

num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 3)

model = model.to(device)

In [16]:
# load the trained model weights that got the best validation accuracy
if torch.cuda.is_available():
    map_location=lambda storage, loc: storage.cuda()
else:
    map_location='cpu'
    
weights = torch.load(TRAINED_MODEL_WEIGHTS_FILE_NAME, map_location=map_location)
model.load_state_dict(weights)

In [17]:
TEST_DATA_PATH = '../data/test'
vanila_transform = transforms.Compose([
                                transforms.Resize(image_size),
                                transforms.CenterCrop(image_size),
                                transforms.ToTensor()
                               ]) 

if not 'datasets' in locals():
    print("create empty datasets")
    datasets = {}
if not 'loaders' in locals():
    print("create empty loaders")

loaders = {}
datasets['test'] = torchvision.datasets.ImageFolder(TEST_DATA_PATH, transform=vanila_transform)
loaders['test'] = torch.utils.data.DataLoader(datasets['test'],
                                          batch_size=1,
                                          shuffle=True,
                                          num_workers=1)


criterion = nn.CrossEntropyLoss()

In [18]:
def test(loaders, model, criterion):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        data, target = data.to(device), target.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
test(loaders, model, criterion)

Test Loss: 1.230233


Test Accuracy: 57% (57/100)


# Competition- Phase 1: generate the result csv file

In [19]:
# # load the model that got the best validation accuracy (uncomment the line below)
# model.load_state_dict(torch.load(TRAINED_MODEL_WEIGHTS_FILE_NAME))

# # list of class names by index, i.e. a name can be accessed like class_names[0]
# class_names = ['A','B','C']

# def predict_class(img_path):
#     # load the image and return the predicted breed
#     img = Image.open(img_path)

#     transform = transforms.Compose([
#                                 transforms.Resize(image_size),
#                                 transforms.CenterCrop(image_size),
#                                 transforms.ToTensor()
#                                ]) 
#     img = transform(img)
#     img = img.unsqueeze(0) 

#     img = Variable(img)

#     img = img.to(device)
        
#     prediction = model(img)  # Returns a Tensor of shape (batch, num class labels)
#     prediction = prediction.data.cpu().numpy().argmax()  # Our prediction will be the index of the class label with the largest value.
#     prediction = class_names[prediction]
#     return prediction 


# predict_class('../data/competition/02186.jpg')

# #Get all test files

# test_results = []

# mango_files = np.array(glob("../data/competition/*"))

# for idx, file in enumerate(mango_files):
#     _ , filename = os.path.split(file)
#     className = predict_class(file)
#     test_results.append([filename, className])
    
# # test_results[:3]

# with open('results.csv', 'w') as f:

#     writer = csv.writer(f)
    
#     for row in test_results:
#         writer.writerow(row)

In [20]:
print("Total run time: {}".format(time.time() - nb_start))

Total run time: 5956.580235958099
